In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
from glob import glob
import cv2
import numpy as np

In [2]:
# Load labeled frames and their corresponding labels
def load_labeled_frames(data_folder):
    images = []
    labels = []

    for image_path in glob(os.path.join(data_folder, '*.jpg')):
        image = cv2.imread(image_path)
        image = cv2.resize(image, (20, 20))

        label = os.path.splitext(os.path.basename(image_path))[0].split('_')[-1]

        images.append(image)

        if label == '0':
            labels.append(0)
        elif label == '1':
            labels.append(1)

    return images, labels

# Preprocess the data
def preprocess_data(images, labels):
    images = np.array(images)
    labels = np.array(labels)

    # Normalize pixel values to be between 0 and 1
    images = images / 255.0

    # Shuffle and split the data into training and testing sets
    images, labels = shuffle(images, labels, random_state=46)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=45)

    return X_train, X_test, y_train, y_test

In [3]:
# Example usage
output_folder = 'labeled_frames_binary_test'

# Load labeled frames and their corresponding labels
images, labels = load_labeled_frames(output_folder)

# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_data(images, labels)

input_shape = X_train[0].shape

In [4]:
y_train

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0])

In [14]:
# Build the model
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid')) 

# Compile the model
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',  # Use binary crossentropy for binary classification
              metrics=['accuracy', 'AUC'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))


Epoch 1/100
1/1 [==============================] - 1s 783ms/step - loss: 0.7120 - accuracy: 0.4737 - auc: 0.0444 - val_loss: 0.6746 - val_accuracy: 0.8000 - val_auc: 0.8333
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 0.6762 - accuracy: 0.6316 - auc: 0.8333 - val_loss: 0.6382 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 0.6486 - accuracy: 0.6316 - auc: 0.9111 - val_loss: 0.6037 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 0.6238 - accuracy: 0.6316 - auc: 0.9111 - val_loss: 0.5704 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 0.6016 - accuracy: 0.6316 - auc: 0.9111 - val_loss: 0.5405 - val_accuracy: 0.8000 - val_auc: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 0.5856 - accuracy: 0.6316 - auc: 0.9111 - val_loss: 0.5157 - val_ac

In [6]:
# Evaluate the model
test_loss, test_acc, test_auc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest AUC:', test_auc)

1/1 - 0s - loss: 0.4312 - accuracy: 0.8000 - auc: 1.0000 - 21ms/epoch - 21ms/step

Test AUC: 1.0


In [15]:
# Number of blinking lights: 4
# x1, y1, w1, h1
# (1374, 773, 27, 27)
# (458, 283, 27, 27)
# (1403, 280, 23, 24)
# (504, 824, 23, 22)

import cv2

# Open the video file
video_path = './vidscapstone/vid9.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

x1, y1, w1, h1 = (1374, 773, 27, 27)
x2, y2, w2, h2 = (458, 283, 27, 27)
x3, y3, w3, h3 = (1403, 280, 23, 24)
x4, y4, w4, h4 = (504, 824, 23, 22)

# Loop through frames
while True:
    # Read a frame
    ret, frame = cap.read()

    # Break the loop if the video is finished
    if not ret:
        break

    # cut out the region
    roi = frame[y1:y1+h1, x1:x1+w1]
    roi2 = frame[y2:y2+h2, x2:x2+w2]
    roi3 = frame[y3:y3+h3, x3:x3+w3]
    roi4 = frame[y4:y4+h4, x4:x4+w4]

    # resize the region to 100x100
    roi = cv2.resize(roi, (20, 20))
    roi2 = cv2.resize(roi2, (20, 20))
    roi3 = cv2.resize(roi3, (20, 20))
    roi4 = cv2.resize(roi4, (20, 20))

    # predict the label
    roi = np.array(roi)
    roi2 = np.array(roi2)
    roi3 = np.array(roi3)
    roi4 = np.array(roi4)

    roi = roi / 255.0
    roi2 = roi2 / 255.0
    roi3 = roi3 / 255.0
    roi4 = roi4 / 255.0

    roi = np.expand_dims(roi, axis=0)
    roi2 = np.expand_dims(roi2, axis=0)
    roi3 = np.expand_dims(roi3, axis=0)
    roi4 = np.expand_dims(roi4, axis=0)

    pred = model.predict(roi)
    pred2 = model.predict(roi2)
    pred3 = model.predict(roi3)
    pred4 = model.predict(roi4)
    
    print(pred)
    print(pred2)
    print(pred3)
    print(pred4)

    # Process the frame (you can add your own processing logic here)
    # For example, you can display the frame using imshow
    cv2.imshow('Frame', frame)

    # Break the loop if the user presses 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    
# Release the video capture object
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step
[[1.]]
[[1.]]
[[0.99999994]]
[[0.00157864]]
1/1 [==============================] - 0s 16ms/step
[[0.99999654]]
[[1.]]
[[1.]]
[[0.99999994]]
1/1 [==============================] - 0s 16ms/step
[[0.9999966]]
[[1.]]
[[0.00157864]]
[[1.]]
1/1 [==============================] - 0s 16ms/step
[[0.99999654]]
[[0.99999994]]
[[0.03754858]]
[[1.]]
1/1 [==============================] - 0s 16ms/step
[[0.00157864]]
[[0.99999994]]
[[0.88321376]]
[[1.]]
1/1 [==============================] - 0s 16ms/step
[[0.00079567]]
[[0.99999994]]
[[0.02681126]]
[[0.9999476]]
1/1 [==============================] - 0s 16ms/step
[[0.00934189]]
[[0.00157864]]
[[0.02849157]]
[[0.99994844]]
1/1 [==============================] - 0s 17ms/step
[[0.00939706]]
[[0.00157864]]
[[8.1142396e-05]]
[[0.00157864]]
1/1 [==============================] - 0s 14ms/step
[[0.00016681]]
[[0.04341068]]
[[8.2534614e-05]]
[[3.8127095e-05]]
1/1 [==============================] - 0s 17ms/st

In [ ]:
import pickle

directory = 'testing_binary_class_classification/'

model.save(directory + 'testing_binary_class_classification_model_Adam.h5')

with open(directory + 'testing_binary_class_classification_model_Adam_history', 'wb') as file_pi:
    pickle.dump(history, file_pi)

with open(directory + 'testing_binary_class_classification_model_Adam_X_test', 'wb') as file_pi:
    pickle.dump(X_test, file_pi)

with open(directory + 'testing_binary_class_classification_model_Adam_y_test', 'wb') as file_pi:
    pickle.dump(y_test, file_pi)